In [2]:
import argparse
import boto3
import pandas as pd
import s3fs
import numpy as np 
import io 
import pyarrow as pa 
import pyarrow.parquet as pq
import duckdb
import os

In [2]:
fs = s3fs.S3FileSystem(profile='AdministratorAccess-769392325318')
paths = fs.glob('mortgage-data-refined/light_transformations/hmda/2018/*.parquet')
duckdb_paths = [f"s3://{p}" for p in paths]

In [3]:
session = boto3.Session(profile_name='AdministratorAccess-769392325318')
creds = session.get_credentials().get_frozen_credentials()

In [4]:
s3 = session.client('s3')
_ = s3.list_objects_v2(Bucket='mortgage-data-refined', MaxKeys=1)

In [5]:
def build_canonical_schema(canonical_types):
    """
    Returns a pyarrow schema object with desired column names and types.
    """
    type_map = {
        "string": pa.string(),
        "float64": pa.float64(),
        "int64": pa.int64()
    }
    return pa.schema([(col, type_map[dtype]) for col, dtype in canonical_types.items()])


In [3]:
# Connect to the database
con = duckdb.connect("hmda.duckdb")

In [6]:
def align_table_to_schema(table, canonical_schema, file_path=None, diagnostics=None):
    """
    Aligns a PyArrow table to the canonical schema.
    Tracks and logs any type mismatches, failed casts, or missing columns.
    """
    arrays = []
    table_column_names = table.schema.names

    for field in canonical_schema:
        col_name = field.name
        desired_type = field.type
        issue = None

        if col_name in table_column_names:
            current_col = table.column(col_name)
            current_type = table.schema.field(col_name).type

            # Force _category/_bucketed suffixes to string
            if col_name.endswith("_category") or col_name.endswith("_bucketed"):
                if not pa.types.is_string(current_type):
                    try:
                        current_col = current_col.cast(pa.string())
                        issue = "cast_to_string_due_to_suffix"
                    except Exception:
                        current_col = pa.nulls(len(table), type=pa.string())
                        issue = "failed_cast_to_string_due_to_suffix"
            elif str(current_type) != str(desired_type):
                try:
                    current_col = current_col.cast(desired_type)
                    issue = f"cast_{current_type}_to_{desired_type}"
                except Exception:
                    current_col = pa.nulls(len(table), type=desired_type)
                    issue = f"failed_cast_{current_type}_to_{desired_type}"

            arrays.append(current_col)

        else:
            # Missing column
            fill_type = (
                pa.string() if col_name.endswith("_category") or col_name.endswith("_bucketed")
                else desired_type
            )
            arrays.append(pa.nulls(len(table), type=fill_type))
            issue = f"missing_column_filled_with_nulls"

        if diagnostics is not None and issue:
            diagnostics.append({
                "column": col_name,
                "expected_type": str(desired_type),
                "issue": issue,
                "file": file_path
            })

    return pa.Table.from_arrays(arrays, schema=canonical_schema)


In [7]:
import pyarrow as pa
import pyarrow.parquet as pq
import s3fs
import pandas as pd
import io
from collections import defaultdict

def normalize_arrow_schema(schema):
    return {field.name: str(field.type) for field in schema}

def coerce_to_schema(df, canonical_schema):
    for col, expected_type in canonical_schema.items():
        if col not in df.columns:
            df[col] = pd.NA
        try:
            if expected_type.startswith("int"):
                df[col] = pd.to_numeric(df[col], errors='coerce').astype("Int64")
            elif expected_type.startswith("float"):
                df[col] = pd.to_numeric(df[col], errors='coerce').astype("float64")
            elif expected_type == "string":
                df[col] = df[col].astype(str)
            else:
                df[col] = df[col].astype(expected_type)
        except Exception:
            df[col] = pd.NA
    return df[list(canonical_schema.keys())]

def verify_parquet_schemas(s3_prefix, profile=None, verbose=True, auto_fix=False, dry_run=False):
    fs = s3fs.S3FileSystem(profile=profile)
    bucket, prefix = s3_prefix.split('/', 1)
    files = fs.glob(f"{bucket}/{prefix}/*.parquet")

    canonical_schema = None
    mismatches = defaultdict(list)

    for file in files:
        with fs.open(file, 'rb') as f:
            schema = pq.read_schema(f)
            norm_schema = normalize_arrow_schema(schema)

        if canonical_schema is None:
            canonical_schema = norm_schema
            continue

        file_mismatch = False
        for col, dtype in norm_schema.items():
            if col not in canonical_schema:
                mismatches[file].append((col, "Unexpected", dtype))
                file_mismatch = True
            elif canonical_schema[col] != dtype:
                mismatches[file].append((col, "TypeMismatch", dtype, canonical_schema[col]))
                file_mismatch = True
        for col in canonical_schema:
            if col not in norm_schema:
                mismatches[file].append((col, "Missing", canonical_schema[col]))
                file_mismatch = True

        if file_mismatch and auto_fix:
            if dry_run:
                print(f"🧪 Would fix: {file}")
            else:
                try:
                    with fs.open(file, 'rb') as f:
                        df = pq.read_table(f).to_pandas()
                    df = coerce_to_schema(df, canonical_schema)
                    table = pa.Table.from_pandas(df)
                    buf = io.BytesIO()
                    pq.write_table(table, buf)
                    fs.pipe(file, buf.getvalue())
                    print(f"🔁 Fixed and overwrote: {file}")
                except Exception as e:
                    print(f"❌ Failed to fix {file}: {e}")

    if mismatches:
        if verbose:
            print("\n❌ Schema mismatches found:")
            for file, issues in mismatches.items():
                print(f"- {file}")
                for issue in issues:
                    if issue[1] == "Unexpected":
                        print(f"    ➕ Unexpected column: {issue[0]} ({issue[2]})")
                    elif issue[1] == "Missing":
                        print(f"    ➖ Missing column: {issue[0]} (expected {issue[2]})")
                    elif issue[1] == "TypeMismatch":
                        print(f"    🔁 Type mismatch: {issue[0]} ({issue[2]} vs {issue[3]})")
        return False, canonical_schema

    print("✅ All files have consistent schema.")
    return True, canonical_schema


In [11]:
def arrow_to_duckdb_type(pa_type):
    if pa.types.is_string(pa_type):
        return "VARCHAR"
    elif pa.types.is_integer(pa_type):
        return "BIGINT"
    elif pa.types.is_floating(pa_type):
        return "DOUBLE"
    elif pa.types.is_boolean(pa_type):
        return "BOOLEAN"
    elif pa.types.is_timestamp(pa_type):
        return "TIMESTAMP"
    elif pa.types.is_date(pa_type):
        return "DATE"
    else:
        return "VARCHAR"

def generate_schema_across_years(base_s3_prefix, table_name, profile=None, years=range(2018, 2024), files_per_year=5):
    import pyarrow as pa
    import pyarrow.parquet as pq
    import s3fs

    def normalize_schema(schema: pa.Schema) -> pa.Schema:
        new_fields = []
        for field in schema:
            name = field.name
            dtype = field.type

            # Normalize known inconsistent fields
            if name in {"total_units"}: 
                new_fields.append(pa.field(name, pa.string()))
            elif name in {"income", "loan_to_value_ratio"}:
                new_fields.append(pa.field(name, pa.float64()))
            elif name.endswith("_category") or name.endswith("_bucketed"):
                new_fields.append(pa.field(name, pa.string()))
            else:
                new_fields.append(field)

        return pa.schema(new_fields)

    fs = s3fs.S3FileSystem(profile=profile)
    schemas = []

    for year in years:
        path_prefix = f"{base_s3_prefix}/{year}"
        bucket, prefix = path_prefix.split("/", 1)
        try:
            files = fs.glob(f"{bucket}/{prefix}/*.parquet")[:files_per_year]
            for path in files:
                with fs.open(path, "rb") as f:
                    raw_schema = pq.read_schema(f)
                    coerced_schema = normalize_schema(raw_schema)
                    schemas.append(coerced_schema)
        except Exception as e:
            print(f"⚠️ Skipping {year} file {path}: {e}")

    if not schemas:
        raise ValueError("❌ No valid schemas found across years.")

    unified_schema = pa.unify_schemas(schemas)

    col_defs = []
    for field in unified_schema:
        duck_type = arrow_to_duckdb_type(field.type)
        col_defs.append(f'"{field.name}" {duck_type}')

    create_stmt = f'CREATE TABLE {table_name} (\n  ' + ',\n  '.join(col_defs) + '\n);'
    return create_stmt


In [12]:
stmt = generate_schema_across_years(
    base_s3_prefix="mortgage-data-refined/light_transformations/hmda",
    table_name="hmda_raw",
    profile="AdministratorAccess-769392325318", 
    files_per_year=30
)

print(stmt)

CREATE TABLE hmda_raw (
  "activity_year" BIGINT,
  "lei" VARCHAR,
  "derived_msa_md" VARCHAR,
  "state_code" VARCHAR,
  "county_code" VARCHAR,
  "census_tract" DOUBLE,
  "conforming_loan_limit" BIGINT,
  "derived_loan_product_type" VARCHAR,
  "derived_dwelling_category" VARCHAR,
  "derived_ethnicity" VARCHAR,
  "derived_race" VARCHAR,
  "derived_sex" VARCHAR,
  "action_taken" DOUBLE,
  "purchaser_type" DOUBLE,
  "preapproval" DOUBLE,
  "loan_type" DOUBLE,
  "loan_purpose" DOUBLE,
  "lien_status" DOUBLE,
  "reverse_mortgage" DOUBLE,
  "open_end_line_of_credit" DOUBLE,
  "business_or_commercial_purpose" DOUBLE,
  "loan_amount" DOUBLE,
  "loan_to_value_ratio" DOUBLE,
  "interest_rate" DOUBLE,
  "rate_spread" DOUBLE,
  "hoepa_status" DOUBLE,
  "total_loan_costs" DOUBLE,
  "total_points_and_fees" DOUBLE,
  "origination_charges" DOUBLE,
  "discount_points" DOUBLE,
  "lender_credits" DOUBLE,
  "loan_term" BIGINT,
  "prepayment_penalty_term" BIGINT,
  "intro_rate_period" BIGINT,
  "negative_a

In [13]:
import pyarrow.parquet as pq
import s3fs

def safe_insert_file(con, path, table_name, profile=None):
    fs = s3fs.S3FileSystem(profile=profile)
    with fs.open(path.replace("s3://", ""), "rb") as f:
        table = pq.read_table(f)
        df = table.to_pandas()
        con.register("temp_df", df)
        con.execute(f"INSERT INTO {table_name} SELECT * FROM temp_df")
        con.unregister("temp_df")


In [14]:
# Start DuckDB
con.execute("INSTALL httpfs; LOAD httpfs")

# Set credentials into DuckDB
con.execute(f"""
    SET s3_region='us-east-1';
    SET s3_access_key_id='{creds.access_key}';
    SET s3_secret_access_key='{creds.secret_key}';
    SET s3_session_token='{creds.token}';
""")

In [15]:
con.execute("Drop table hmda_raw")
con.execute(stmt)

In [16]:
import pyarrow.parquet as pq
import pandas as pd
import s3fs

def insert_parquet_files_into_duckdb(con, table_name, s3_prefix, profile=None, max_files=None, file_filter=None):
    """
    Read parquet files via PyArrow from S3, register in DuckDB, and insert into table.

    Parameters:
        con (duckdb.DuckDBPyConnection): Active DuckDB connection
        table_name (str): Target DuckDB table
        s3_prefix (str): Format 'bucket-name/path/to/files'
        profile (str): Optional AWS profile for s3fs
        max_files (int): Limit how many files to process
        file_filter (callable): Optional filtering function on file paths
    """
    fs = s3fs.S3FileSystem(profile=profile)
    bucket, prefix = s3_prefix.split("/", 1)
    files = fs.glob(f"{bucket}/{prefix}/*.parquet")
    s3_paths = [f"s3://{f}" for f in files]

    if file_filter:
        s3_paths = list(filter(file_filter, s3_paths))
    if max_files:
        s3_paths = s3_paths[:max_files]

    print(f"📦 Found {len(s3_paths)} files to insert into {table_name}")

    for path in s3_paths:
        print(f"🔄 Reading and inserting {path}")
        try:
            with fs.open(path.replace("s3://", ""), "rb") as f:
                table = pq.read_table(f)
                df = table.to_pandas()
                if "combined_laon_to_value_ratio" in df.columns:
                    df.rename(columns={"combined_laon_to_value_ratio": "combined_loan_to_value_ratio"}, inplace=True)

                # Align to expected schema
                expected_cols = con.execute("PRAGMA table_info('hmda_raw')").fetchdf()["name"].tolist()
                for col in expected_cols:
                    if col not in df.columns:
                        df[col] = pd.NA
                df = df[expected_cols]
            con.register("temp_df", df)
            con.execute(f"INSERT INTO {table_name} SELECT * FROM temp_df")
            con.unregister("temp_df")
        except Exception as e:
            print(f"❌ Failed to insert {path}: {e}")


In [27]:
insert_parquet_files_into_duckdb(
    con,
    table_name="hmda_raw",
    s3_prefix="mortgage-data-refined/light_transformations/hmda/2018",
    profile="AdministratorAccess-769392325318", 
    max_files=None
)

📦 Found 61 files to insert into hmda_raw
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part0_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part10_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part11_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part12_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part13_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part14_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part15_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part16_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part17_0.parquet
🔄 Reading and i

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part1_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part20_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part21_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part22_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part23_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part24_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part25_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part26_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part27_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part42_0.parquet


...

🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part52_0.parquet


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part53_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part54_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part55_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part56_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part57_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part58_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part59_0.parquet


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part5_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part60_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part6_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part7_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part8_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part9_0.parquet


In [17]:
for year in range(2018, 2024): 
    print(f"Processing {year}")
    insert_parquet_files_into_duckdb(
        con,
        table_name="hmda_raw",
        s3_prefix=f"mortgage-data-refined/light_transformations/hmda/{year}",
        profile="AdministratorAccess-769392325318", 
        max_files=None
    )

Processing 2018
📦 Found 61 files to insert into hmda_raw
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part0_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part10_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part11_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part12_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part13_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part14_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part15_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part16_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part17_0.parquet

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part46_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part47_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part48_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part49_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part4_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part50_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part51_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part52_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2018/part_part53_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2019/part_part14_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2019/part_part15_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2019/part_part16_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2019/part_part17_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2019/part_part18_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2019/part_part19_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2019/part_part1_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2019/part_part20_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2019/part_part21_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light

...

🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part27_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part28_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part29_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part2_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part30_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part31_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part32_0.parquet


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part33_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part34_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part35_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part36_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part37_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part38_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part39_0.parquet


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part3_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part40_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part41_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part42_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part43_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part44_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part45_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part4_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_transformations/hmda/2023/part_part5_0.parquet
🔄 Reading and inserting s3://mortgage-data-refined/light_t

In [18]:
def execute_sql_return_df(sql_text): 
    df = con.execute(sql_text).fetchdf()
    return df

In [5]:
def execute_sql_print_result(sql_text): 
    results = con.execute(sql_text).fetchall()
    print(results)

In [7]:
text = '''
show all tables
'''
execute_sql_print_result(text)

[]


In [8]:
con.close()